# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('City_data.csv')

city_data

,Unnamed: 0,City,Lat,Lng,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bondoukou,8.03,-2.80,88.59,55,50,2.73,CI,1591633160
1,1,cap malheureux,-19.98,57.61,70.00,68,0,5.99,MU,1591633161
2,2,lebu,-37.62,-73.65,53.71,83,94,5.26,CL,1591633161
3,3,albany,42.60,-73.97,73.99,35,49,10.02,US,1591632884
4,4,tasiilaq,65.61,-37.64,41.00,85,75,24.16,GL,1591633161
...,...,...,...,...,...,...,...,...,...,...
523,523,mercedes,-34.65,-59.43,66.99,66,57,3.00,AR,1591633310
524,524,barentu,15.11,37.59,93.94,21,47,11.25,ER,1591633310
525,525,port blair,11.67,92.75,81.07,82,100,16.51,IN,1591633310
526,526,bambanglipuro,-7.95,110.28,77.74,87,63,10.31,ID,1591633180


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locts = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locts, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=8)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacations_df=city_data[(city_data["Max Temperature (F)"] >= 70) & (city_data['Max Temperature (F)'] <= 80) & (city_data['Wind Speed'] < 10) &
                   (city_data['Cloudiness'] < 20) & (city_data['Humidity'] < 50) ]
vacations_df



,Unnamed: 0,City,Lat,Lng,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date
8,8,oshnaviyeh,37.04,45.10,77.00,33,0,4.70,IR,1591633163
65,65,cabo san lucas,22.89,-109.91,78.80,47,5,3.36,MX,1591633166
104,104,javanrud,34.80,46.52,70.05,27,0,1.12,IR,1591633196
146,146,east london,-33.02,27.91,72.18,44,0,9.60,ZA,1591632916
197,197,opuwo,-18.06,13.84,76.48,19,0,6.87,NaN,1591633221
207,207,roma,43.21,-75.46,71.60,43,1,6.93,US,1591632965
278,278,culpeper,38.50,-77.97,79.00,41,1,5.82,US,1591633242
283,283,kalabo,-14.97,22.68,71.42,32,0,3.87,ZM,1591633244
392,392,keetmanshoop,-26.58,18.13,78.80,15,0,6.93,NaN,1591633274
425,425,priego de cordoba,37.44,-4.20,77.00,31,0,3.00,ES,1591633282


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
vacations_df.loc[:,"Hotel Name"]=""
vacations_df

C:\Users\aldo_\anaconda3\envs\Boot\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\aldo_\anaconda3\envs\Boot\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Unnamed: 0,City,Lat,Lng,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,oshnaviyeh,37.04,45.10,77.00,33,0,4.70,IR,1591633163,
65,65,cabo san lucas,22.89,-109.91,78.80,47,5,3.36,MX,1591633166,
104,104,javanrud,34.80,46.52,70.05,27,0,1.12,IR,1591633196,
146,146,east london,-33.02,27.91,72.18,44,0,9.60,ZA,1591632916,
197,197,opuwo,-18.06,13.84,76.48,19,0,6.87,NaN,1591633221,
207,207,roma,43.21,-75.46,71.60,43,1,6.93,US,1591632965,
278,278,culpeper,38.50,-77.97,79.00,41,1,5.82,US,1591633242,
283,283,kalabo,-14.97,22.68,71.42,32,0,3.87,ZM,1591633244,
392,392,keetmanshoop,-26.58,18.13,78.80,15,0,6.93,NaN,1591633274,
425,425,priego de cordoba,37.44,-4.20,77.00,31,0,3.00,ES,1591633282,


In [6]:
hotel_df=[]


# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in vacations_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    

    #Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        vacations_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.append(name_address["results"][0]["name"])
        print(f"Hotel found nearby. {index}")
    except (KeyError, IndexError):
        print(f"Missing field/result... skipping. {index}")

Missing field/result... skipping. 8


C:\Users\aldo_\anaconda3\envs\Boot\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel found nearby. 65
Hotel found nearby. 104
Hotel found nearby. 146
Hotel found nearby. 197
Hotel found nearby. 207
Hotel found nearby. 278
Missing field/result... skipping. 283
Hotel found nearby. 392
Hotel found nearby. 425


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacations_df.iterrows()]
locations = vacations_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Add the layer to the map
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))